# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
!jupyter nbextension enable --py gmaps 
# gmaps.configure(api_key=g_key)
# pip install gmaps
# %matplotlib ipympl

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
inpath='../output_data/cities.csv'
cities_df= pd.read_csv(inpath)

gmaps.configure(api_key=g_key)

In [4]:
cities_df.head()

,Unnamed: 0,Cities,Country,Date,Cloudiness,Humidity,Latitude,Longtitude,Max Temperature,Windspeed
0,0,Port-Cartier,CA,1598385468,90,100,50.03,-66.87,59.00,6.93
1,1,Butaritari,KI,1598385468,88,66,3.07,172.79,81.73,5.21
2,2,Klaksvík,FO,1598385468,89,81,62.23,-6.59,46.40,4.70
3,3,Sechura,PE,1598385469,34,48,-5.56,-80.82,73.85,16.02
4,4,Belaya Gora,RU,1598385469,1,80,68.53,146.42,36.57,2.15


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
locations = cities_df[["Latitude", "Longtitude"]].astype(float)
humidity = cities_df["Humidity"].astype(float)


In [6]:
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(layout=figure_layout)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df= cities_df.copy()
hotel_df['Hotel Name'] = ''
hotel_df.head()

,Unnamed: 0,Cities,Country,Date,Cloudiness,Humidity,Latitude,Longtitude,Max Temperature,Windspeed,Hotel Name
0,0,Port-Cartier,CA,1598385468,90,100,50.03,-66.87,59.00,6.93,
1,1,Butaritari,KI,1598385468,88,66,3.07,172.79,81.73,5.21,
2,2,Klaksvík,FO,1598385468,89,81,62.23,-6.59,46.40,4.70,
3,3,Sechura,PE,1598385469,34,48,-5.56,-80.82,73.85,16.02,
4,4,Belaya Gora,RU,1598385469,1,80,68.53,146.42,36.57,2.15,


In [8]:
# A max temperature lower than 80 degrees but higher than 70.

hotel_df= hotel_df.loc[70<hotel_df['Max Temperature'],:] 
hotel_df= hotel_df.loc[hotel_df['Max Temperature']< 80,:]
# Zero cloudiness.

hotel_df= hotel_df.loc[hotel_df['Cloudiness'] == 0,:]
# Wind speed less than 10 mph.
hotel_df= hotel_df.loc[hotel_df['Windspeed'] <10,:]
# & (hotel_df['Max Temperature'])<80

hotel_df

,Unnamed: 0,Cities,Country,Date,Cloudiness,Humidity,Latitude,Longtitude,Max Temperature,Windspeed,Hotel Name
71,71,Saurimo,AO,1598385476,0,30,-9.66,20.39,70.11,1.72,
110,110,Nanzhang,CN,1598385480,0,71,31.78,111.84,73.65,4.09,
125,125,Kapustin Yar,RU,1598385482,0,29,48.58,45.74,70.84,6.42,
139,139,Yamada,JP,1598385483,0,85,39.47,141.95,71.06,2.91,
173,173,Lata,PT,1598385487,0,80,40.16,-8.33,77.00,5.01,
200,200,Pemba,MZ,1598385490,0,80,-12.97,40.52,74.41,8.59,
223,223,Tūkrah,LY,1598385492,0,78,32.53,20.58,77.45,6.33,
261,261,Nador,MA,1598385408,0,83,35.17,-2.93,78.80,2.24,
351,351,Angoche,MZ,1598385505,0,81,-16.23,39.91,73.33,9.78,
359,359,Zhangye,CN,1598385506,0,24,38.93,100.45,73.22,8.10,


In [9]:
hotel_name=[]
for row in hotel_df.index:
    try:
        url_h= f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location='+f'{hotel_df["Latitude"][row]},{hotel_df["Longtitude"][row]}'+f'&radius=5000&type=hotel&&keyword=lodging&key={g_key}'
        response = requests.get(url_h).json()
        hotel_name.append(response['results'][0]['name'])
    except:
        url_h= f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location='+f'{hotel_df["Latitude"][row]},{hotel_df["Longtitude"][row]}'+f'&radius=5000&type=hotel&key={g_key}'
        response = requests.get(url_h).json()
        hotel_name.append(response['results'][1]['name'])

In [10]:
hotel_df['Hotel Name']=hotel_name

In [12]:
# response['results'][4]['types']
hotel_name

['iu Hotel Saurimo Rotunda',
 'Fengyuan International Hotel',
 'Gostinitsa "Znamensk"',
 'Hotel Business Inn Yamada',
 'Quinta do Outeiro',
 'Pemba Beach Resort Hotel',
 'استراحة لاقروتا',
 'Hotel Marchica Lagoon Resort, Nador Morocco',
 'Guest House HESADA',
 'Holiday Inn Express Zhangye',
 'B&B Donna Ida 1900',
 'Latitude20 The Dunes Apartments',
 'Residência da Esperança Zezinha Sonhe',
 'Nuojin Hotel',
 'Altınorfoz',
 'Agriturismo Helianthus',
 'B&B NATURE SPA CHILCHES']

In [13]:
hotel_df=hotel_df.rename(columns={'Cities':'City', 'Latitude': 'Lat','Longtitude': 'Lng' })
hotel_df

,Unnamed: 0,City,Country,Date,Cloudiness,Humidity,Lat,Lng,Max Temperature,Windspeed,Hotel Name
71,71,Saurimo,AO,1598385476,0,30,-9.66,20.39,70.11,1.72,iu Hotel Saurimo Rotunda
110,110,Nanzhang,CN,1598385480,0,71,31.78,111.84,73.65,4.09,Fengyuan International Hotel
125,125,Kapustin Yar,RU,1598385482,0,29,48.58,45.74,70.84,6.42,"Gostinitsa ""Znamensk"""
139,139,Yamada,JP,1598385483,0,85,39.47,141.95,71.06,2.91,Hotel Business Inn Yamada
173,173,Lata,PT,1598385487,0,80,40.16,-8.33,77.00,5.01,Quinta do Outeiro
200,200,Pemba,MZ,1598385490,0,80,-12.97,40.52,74.41,8.59,Pemba Beach Resort Hotel
223,223,Tūkrah,LY,1598385492,0,78,32.53,20.58,77.45,6.33,استراحة لاقروتا
261,261,Nador,MA,1598385408,0,83,35.17,-2.93,78.80,2.24,"Hotel Marchica Lagoon Resort, Nador Morocco"
351,351,Angoche,MZ,1598385505,0,81,-16.23,39.91,73.33,9.78,Guest House HESADA
359,359,Zhangye,CN,1598385506,0,24,38.93,100.45,73.22,8.10,Holiday Inn Express Zhangye


In [14]:
#     url_h= f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?
# location='+f'{hotel_df["Latitude"][row]},{hotel_df["Longtitude"][row]}'+f'&radius=5000&
# type=hotel&&key={g_key}'\

#URL SAID TYPE = HOTEL

# response['results'][0] bring up the political locality, and a horse ride

# response['results'][1] bring up HOTELS. A RESTAURANT, A Farm in Spain, A Minqin Graves Hall in China, 
# A Town Hall in Japan, A Bureau Labour Union Committee in China, A sub-city in China

#URL SAID TYPE = HOTEL

# response['results'][2] bring up MORE HOTELS. A Petrol station in LY, A Factory in Japan, A Health Bureau in China,
# A sub-city in China, A bank in Angola, A Truck Corporation in Turkey. 

#URL SAID TYPE = HOTEL

#GOOGLE DO NOT STORE GOOD DATA IN CHINA. or JAPAN. HOW?

#URL SAID TYPE = HOTEL & KEY WORD= Lodging:

# WorkBetter FOR ALMOST ALL OF THEM  but somehow failed to fetch data with EXACT LODGING INSIDE a few (Lata 	PT) had to use the old link for it


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [16]:
# hotel_info

In [17]:
# Add marker layer ontop of heat map


# Display figure
fig2 = gmaps.figure(layout=figure_layout)

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig2.add_layer(markers)

fig2

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…